In [1]:
!pip install crewai

In [2]:
!pip install langchain-openai
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

  Using cached openai-2.23.0-py3-none-any.whl.metadata (29 kB)
Using cached openai-2.23.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: openai
    Found existing installation: openai 1.83.0
    Uninstalling openai-1.83.0:
      Successfully uninstalled openai-1.83.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 1.9.3 requires openai~=1.83.0, but you have openai 2.23.0 which is incompatible.
instructor 1.12.0 requires openai<2.0.0,>=1.70.0, but you have openai 2.23.0 which is incompatible.


In [4]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:

# ---------------------------------------
# 1. LLM Configuration
# ---------------------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2
)

# ---------------------------------------
# 2. Agents
# ---------------------------------------

# Agent 1 – Troubleshooter
troubleshooter = Agent(
    name="Network Troubleshooter",
    role="Diagnose Wi-Fi and network issues",
    goal="Identify symptoms, check context, detect potential problems",
    backstory=(
        "You are an experienced home network troubleshooter specializing "
        "in identifying issues like Wi-Fi interference, signal drop, congestion, "
        "device overload, hardware faults, and ISP problems."
    ),
    llm=llm,
)

# Agent 2 – Analyzer
analyzer = Agent(
    name="Network Analyzer",
    role="Analyze technical root causes",
    goal="Analyze the troubleshooter findings and determine root cause",
    backstory=(
        'You are a senior network engineer who studies signals, frequencies, '
        'router distance, channel congestion, firmware issues, ISP reliability, '
        'and device load to find the root cause of the problem.'
    ),
    llm=llm,
)

# Agent 3 – Recommender
recommender = Agent(
    name="Network Recommender",
    role="Recommend optimized solutions",
    goal="Provide precise and actionable recommendations to fix the issue",
    backstory=(
        "You are an expert at suggesting clear, step-by-step remedies for "
        "Wi-Fi issues, such as router placement, channel selection, "
        "firmware updates, mesh network usage, and interference mitigation."
    ),
    llm=llm,
)

# ---------------------------------------
# 3. Tasks
# ---------------------------------------
task1 = Task(
    description=(
        "A user reports: 'My Wi-Fi is very slow in the living room.' "
        "Diagnose the symptoms and list possible issues."
    ),
    agent=troubleshooter,
    expected_output="A list of 3–5 suspected networking issues."
)

task2 = Task(
    description=(
        "Analyze the troubleshooter's list and determine the most likely root cause. "
        "Consider signal strength, router position, obstacles, ISP issues, congestion, "
        "channel overlap, interference, and device load."
    ),
    agent=analyzer,
    expected_output="One clearly identified root cause with reasoning."
)

task3 = Task(
    description=(
        "Provide 3–5 clear, actionable steps to fix the identified root cause. "
        "Keep instructions simple for home users."
    ),
    agent=recommender,
    expected_output="A list of recommended solutions."
)

# ---------------------------------------
# 4. Crew
# ---------------------------------------
crew = Crew(
    agents=[troubleshooter, analyzer, recommender],
    tasks=[task1, task2, task3],
    process="sequential",
)

# ---------------------------------------
# 5. Execute
# ---------------------------------------
if __name__ == "__main__":
    result = crew.kickoff()
    print("\n\n=== FINAL HOME NETWORKING SOLUTION ===\n")
    print(result)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable